<a href="https://colab.research.google.com/github/Ephuyo/ChatBot_Informatica_Unsaac/blob/main/YachayBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalar la biblioteca 'unidecode' para realizar la transliteración de caracteres Unicode a caracteres ASCII
!pip install unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 3.8 MB/s eta 0:00:00


In [ ]:
# ----------------------------------------------------Importar las bibliotecas necesarias----------------------------------------------------------------------
# Para manejar archivos JSON
import json
# Biblioteca de procesamiento de lenguaje natural
import nltk
# Para remover acentos y caracteres especiales de texto
import unidecode
# Biblioteca de procesamiento de lenguaje natural avanzado
import spacy
# Función para tokenizar palabras
from nltk.tokenize import word_tokenize
# Lista de palabras detenidas (stop words)
from nltk.corpus import stopwords
# Para crear vectores TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
# Para calcular similitud de coseno
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
# Descargar los recursos necesarios de NLTK
nltk.download('punkt')  # Descargar el tokenizador de NLTK
nltk.download('stopwords')  # Descargar la lista de stop words en español

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# Actualiza la biblioteca 'spacy' a la última versión disponible
!pip install -U spacy

# Descarga el modelo de procesamiento de lenguaje natural en español ("es_core_news_sm") de spaCy
!python -m spacy download es_core_news_sm

In [ ]:
# Cargar el modelo de lenguaje en español de spaCy
nlp = spacy.load('es_core_news_sm')

In [ ]:
# Cargar el corpus desde un archivo JSON
with open('corpus.json', 'r', encoding='utf-8') as archivo:
    corpus = json.load(archivo)

In [ ]:
# Definir las palabras detenidas (stop words) en español
palabras_detenidas = set(stopwords.words('spanish'))

In [ ]:
def preprocesar_texto(texto):
    # Procesar el texto con el modelo de spaCy
    doc = nlp(texto)

    # Filtrar los tokens basados en ciertas propiedades
    tokens_filtrados = [
        token.lemma_ for token in doc
        if not token.is_punct             # No es un signo de puntuación
        and not token.is_space            # No es un espacio en blanco
        and not token.is_stop             # No es una palabra de parada (palabra común que se suele filtrar en el procesamiento de lenguaje)
        and not token.is_digit            # No es un dígito
        and not token.like_num            # No se parece a un número (puede ser una combinación de dígitos y letras)
    ]

    # Unir los tokens filtrados en una cadena de texto
    return ' '.join(tokens_filtrados)


In [ ]:
def crear_vectores_tfidf(corpus):
    # Crear una lista para almacenar todas las respuestas del corpus
    todas_las_respuestas = []

    # Recorrer cada sección en el corpus y extender la lista con los patrones de respuesta
    for seccion in corpus.values():
        todas_las_respuestas.extend(seccion['patrones'])

    # Crear un vectorizador TF-IDF con el preprocesamiento de texto definido anteriormente
    vectorizador = TfidfVectorizer(preprocessor=preprocesar_texto)

    # Transformar las respuestas en una matriz TF-IDF utilizando el vectorizador
    matriz_tfidf = vectorizador.fit_transform(todas_las_respuestas)

    # Devolver el vectorizador y la matriz TF-IDF resultante
    return vectorizador, matriz_tfidf

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode

def buscar_patrones_similares(entrada_usuario, vectorizador, matriz_tfidf, corpus):
    # Preprocesar la entrada del usuario utilizando la función preprocesar_texto definida anteriormente
    entrada_usuario_preprocesada = preprocesar_texto(entrada_usuario)

    # Transformar la entrada del usuario en un vector TF-IDF utilizando el vectorizador
    vector_entrada = vectorizador.transform([entrada_usuario_preprocesada])

    # Calcular las puntuaciones de similitud coseno entre el vector de entrada y la matriz TF-IDF
    puntuaciones_similitud = cosine_similarity(vector_entrada, matriz_tfidf)

    # Obtener los índices de las respuestas más similares en orden descendente de similitud
    indices_mas_similares = puntuaciones_similitud.argsort()[0, ::-1]

    # Inicializar una lista para almacenar las respuestas relevantes
    respuestas = []

    # Iterar a través de los índices de respuestas más similares
    for indice in indices_mas_similares:
        similitud = puntuaciones_similitud[0, indice]
        if similitud >= 0.2:
            patrones = []

            # Extender la lista de patrones con los patrones de todas las secciones en el corpus
            for seccion in corpus.values():
                patrones.extend(seccion['patrones'])

            # Obtener el patrón similar basado en el índice actual
            patron_similar = patrones[indice].lower()

            # Convertir el patrón similar a una forma ASCII normalizada (quitar acentos)
            patron_similar = unidecode(patron_similar)

            # Buscar el patrón similar en las secciones del corpus y obtener las respuestas correspondientes
            for clave, valor in corpus.items():
                for patron in valor['patrones']:
                    if unidecode(patron.lower()) == patron_similar:
                        respuestas = valor['respuestas']
                        break
            break

    return respuestas


In [ ]:
import random

In [ ]:
SALUDOS_INPUTS = ("hola","buenas", "saludos", "qué tal", "hey", "buenos dias", "Buenos dias", "Buenas", "buenas", "hola, que tal?", "como estas?")
SALUDOS_OUTPUTS = ["Hola", "Hola, ¿Cómo te puedo ayudar?", "Hola, encantado de hablar contigo","¿Hola, en que puedo ayudarte?"]

def saludos(sentence):
  for word in sentence.split():
    if word.lower() in SALUDOS_INPUTS:
      return random.choice(SALUDOS_OUTPUTS)

In [ ]:
def chat():
    print("\n=====================================================================================================================")
    print("\n************************* ¡BIENVENIDO AL CHATBOT DE LA ESCUELA DE INGENIERÍA INFORMÁTICA! ***************************")
    print("************************************************** ¡YACHAYBOT! ******************************************************")
    print("\n=====================================================================================================================\n")
    print(" Puedes escribir 'salir' en cualquier momento para terminar.\n")

    vectorizador, matriz_tfidf = crear_vectores_tfidf(corpus)

    while True:
        entrada_usuario = input("+ Tú: ")
        print()
        r = saludos(entrada_usuario.lower())

        if entrada_usuario.lower() in ['salir', 'adiós', 'chao', 'cerrar','adios','chau', 'hasta pronto','gracias', 'salir']:
            print(">> YachayBot: ¡Hasta luego!")
            break

        respuestas = buscar_patrones_similares(entrada_usuario, vectorizador, matriz_tfidf, corpus)
        if (r != None):
          print(">> YachayBot: " + r)
          print("")
        else:
            if not respuestas:
              print(">> YachayBot: Lo siento, no entiendo tu pregunta.\n")
            else:
                for respuesta in respuestas:
                    print(">> YachayBot:", respuesta)
                print("")

chat()



************************* ¡BIENVENIDO AL CHATBOT DE LA ESCUELA DE INGENIERÍA INFORMÁTICA! ***************************
************************************************** ¡YACHAYBOT! ******************************************************


 Puedes escribir 'salir' en cualquier momento para terminar.

+ Tú: Hola

>> YachayBot: Hola

+ Tú: que tesis estan insnritas

>> YachayBot: - Implementación de un dataset para la evaluación de modelos de análisis de sentimientos en la clasificación de tweets
>> YachayBot: - Aplicación para el reconocimiento de texto y transformación de voz, y desarrollo de un prototipo de impresora braille orientado a personas con discapacidad visual
>> YachayBot: - Análisis comparativo de la performance de los descriptores Wavelet y Fourier, aplicado a la detección de anomalías en trayectorias
>> YachayBot: - Generación automática de letras de canciones usando redes neuronales recurrentes para el quechua
>> YachayBot: - Implementación de un sistema experto para el 